<a href="https://colab.research.google.com/github/sundarp17/semi-supervised/blob/main/train3_svc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import numpy as np
import pandas as pd

In [153]:

train3 = pd.read_csv(r'/content/train_data3.csv')
test = pd.read_csv(r'/content/test_data.csv')

In [154]:

train3['Target'] = train3['Target'].replace(['Others'],'Invalid')


In [155]:
train3['Target'].unique()

array(['Invalid', 'Issue', 'Analysis', 'Facts', 'Conclusion',
       'Rule/Law/Holding'], dtype=object)

In [156]:
#cleaning
import nltk
import re
import string
nltk.download('stopwords')
nltk.download('wordnet')
stopword=nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wl= WordNetLemmatizer()

def clean_text(text):
  text="".join([word.lower() for word in text if word not in string.punctuation])
  tokens = re.split('\W+',text)
  text = [wl.lemmatize(word) for word in tokens if word not in stopword]
  return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [157]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer = clean_text)

train3_Xtfidf = tfidf_vect.fit_transform(train3['Sentence'])


In [158]:
test['Target']=test['Target'].replace(['Others'],'Invalid')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x.lower() for x in str(x).split()))
test['Sentence'] = test['Sentence'].str.replace('[^\w\s]','')
from nltk.corpus import stopwords
words = stopwords.words('english')
test['Sentence'] = test['Sentence'].apply(lambda x: " ".join(x for x in x.split() if x not in words))

In [159]:
unlabel_1 = test.loc[:100]
print("length of unlabel_1",len(unlabel_1))
unlabel_2 = test.loc[101:200]
print("length of unlabel_2",len(unlabel_2))
unlabel_3 = test.loc[201:300]
print("length of unlabel_3",len(unlabel_3))
unlabel_4 = test.loc[301:400]
print("length of unlabel_4",len(unlabel_4))
unlabel_5 = test.loc[401:]
print("length of unlabel_5",len(unlabel_5))

length of unlabel_1 101
length of unlabel_2 100
length of unlabel_3 100
length of unlabel_4 100
length of unlabel_5 120


In [160]:
unlabel_2.reset_index(inplace=True)
del unlabel_2['index']
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [161]:
unlabel_3.reset_index(inplace=True)
del unlabel_3['index']
unlabel_3.head()

,Unnamed: 0,Sentence,Target
0,1882,find evidence amply sufficient sustain juryâs ...,Analysis
1,964,actions show thatâ,Invalid
2,1417,certification attending physician shown filed ...,Facts
3,2932,testified agent bland first left automobile wa...,Facts
4,1584,officer testified could smell odor alcohol app...,Facts


In [162]:
unlabel_4.reset_index(inplace=True)
del unlabel_4['index']
unlabel_4.head()

,Unnamed: 0,Sentence,Target
0,266,section 6 pl,Invalid
1,3469,amended answer appellant alleged bond invalid ...,Analysis
2,1471,evidence sufficient support conviction appella...,Analysis
3,1990,cases cited note 19,Invalid
4,2757,july 8 1960 order entered revoking said probat...,Facts


In [163]:
unlabel_5.reset_index(inplace=True)
del unlabel_5['index']
unlabel_5.head()

,Unnamed: 0,Sentence,Target
0,2979,overrule appellantâs contention court erred pe...,Analysis
1,2135,state regarded authority case,Invalid
2,1591,testimony adduced appellant mother lillie mae ...,Facts
3,658,appellant needle marks inside left elbow,Facts
4,2562,analysis contents 3 capsules chemist kenneth a...,Facts


In [164]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.calibration import CalibratedClassifierCV


X_train, x_val, Y_train, y_val = train_test_split(train3_Xtfidf,train3['Target'],test_size=0.26,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(X_train, Y_train)
pred_svm = model.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6758849557522124
                  precision    recall  f1-score   support

        Analysis       0.52      0.39      0.44       127
      Conclusion       0.75      0.59      0.66        46
           Facts       0.74      0.90      0.82       487
         Invalid       0.48      0.45      0.46       128
           Issue       0.59      0.55      0.57        47
Rule/Law/Holding       0.62      0.19      0.29        69

        accuracy                           0.68       904
       macro avg       0.62      0.51      0.54       904
    weighted avg       0.66      0.68      0.65       904



In [165]:
x_un1 = tfidf_vect.transform(unlabel_1['Sentence'])
pred_unlabel_1 = model.predict_proba(x_un1)

In [166]:
import numpy as np
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_1:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 4, 3, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3]
[0.9914866257895472, 0.9959547657590856, 0.9954988909435185, 0.9999607280731516, 0.9800596109382469, 0.987218326178931, 0.9990850965571149, 0.9870684287489697, 0.9937198849406166, 0.9915491823243745, 0.998782931370616, 0.9962894149875867, 0.9994373158043389, 0.9976401893710839, 0.9805375878100809, 0.9990528824157555, 0.9922843665510279, 0.9995383330937709, 0.991247104745695, 0.9931167028947567, 0.9959792312540363, 0.9970863699263327, 0.9986101569341662, 0.9995639865225433, 0.9994214722788797, 0.9908042140814732, 0.9972877437447277, 0.9835059410160987, 0.9870313683143064, 0.9958199941053751, 0.993565872828943, 0.9929282845097457, 0.9991655779898759, 0.9924509387542724, 0.9908040481804957, 0.9999541316369089, 0.990531792663524, 0.9848563527299808, 0.9994174763656204, 0.9993357679889415, 0.9994127325254235, 0.9966469695046477]
[0, 1, 4, 5, 6, 8, 11, 12, 17, 18, 19

In [167]:
unlabel_1=unlabel_1.loc[[0, 1, 4, 5, 6, 8, 11, 12, 17, 18, 19, 21, 23, 24, 26, 29, 30, 31, 33, 35, 37, 39, 40, 45, 48, 49, 50, 51, 52, 55, 56, 59, 62, 70, 72, 76, 77, 87, 89, 94, 96, 98],:]
unlabel_1.head()

,Unnamed: 0,Sentence,Target
0,2173,darden sr testified âœi saw staggering â prett...,Facts
1,466,additional moral justification may due complai...,Analysis
4,79,eligible leaves renew contacts family,Facts
5,1955,got downstairs ponce went car wife left home,Facts
6,324,defendant argues impossible reconcile plaintif...,Facts


In [168]:
frame_1 = [train3,unlabel_1]
train3_1 = pd.concat(frame_1)
len(train3_1)

3518

In [169]:
x_train_1 = tfidf_vect.transform(train3_1['Sentence'])
x_train_1.shape
X_train, x_val, Y_train, y_val = train_test_split(x_train_1,train3_1['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_1=calibrator.fit(X_train, Y_train)
pred_svm = model_1.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6732954545454546
                  precision    recall  f1-score   support

        Analysis       0.44      0.40      0.42       101
      Conclusion       0.78      0.55      0.65        38
           Facts       0.75      0.90      0.82       358
         Invalid       0.56      0.51      0.54       111
           Issue       0.62      0.50      0.56        40
Rule/Law/Holding       0.61      0.25      0.35        56

        accuracy                           0.67       704
       macro avg       0.63      0.52      0.55       704
    weighted avg       0.66      0.67      0.65       704



In [170]:
x_un2 = tfidf_vect.transform(unlabel_2['Sentence'])
pred_unlabel_2 = model_1.predict_proba(x_un2)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_2:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2]
[0.9874399736671714, 0.9888050410555216, 0.99929201853482, 0.9838849161664384, 0.9944939229934954, 0.9838768506769111, 0.9982199019446389, 0.9989097055135521, 0.9999548372025909, 0.9962792491831406, 0.9825007482500672, 0.9988765715636176, 0.9895420388462405, 0.9823037757258641, 0.9994852762357924, 0.999962176199194, 0.9993797422748021, 0.9995837918661784, 0.9991136883257726, 0.9928972042082626, 0.9986313993380244, 0.9974911885603939, 0.9996122975217177, 0.9995449742184047, 0.9998253511504835, 0.9986447152638035, 0.998854459208351, 0.9935739889023255, 0.9988282159425206, 0.9922416322969718, 0.9916561059678648, 0.998796707151954, 0.9981485298213575, 0.9989662546271566, 0.9868465106366314]
[0, 1, 2, 3, 4, 6, 7, 8, 12, 20, 21, 29, 31, 34, 36, 37, 39, 42, 49, 51, 54, 55, 59, 63, 64, 69, 75, 77, 83, 85, 86, 88, 90, 91, 95]
35
35
35


In [171]:
unlabel_2 = unlabel_2.loc[[0, 1, 2, 3, 4, 6, 7, 8, 12, 20, 21, 29, 31, 34, 36, 37, 39, 42, 49, 51, 54, 55, 59, 63, 64, 69, 75, 77, 83, 85, 86, 88, 90, 91, 95],:]
unlabel_2.head()

,Unnamed: 0,Sentence,Target
0,1207,examination occurred night middle may testifie...,Invalid
1,3400,undisputed testimony reflects appellant drivin...,Facts
2,2072,appellant testify offer evidence behalf,Facts
3,517,56 uniform act regulating traffic highways gis...,Rule/Law/Holding
4,906,agreed accompany gave pawn ticket money reclai...,Facts


In [172]:
frame_2 = [train3_1,unlabel_2]
train3_2 = pd.concat(frame_2)
len(train3_2)

3553

In [173]:
x_train_2 = tfidf_vect.transform(train3_2['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_2,train3_2['Target'],test_size=0.27,random_state=2)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_2=calibrator.fit(X_train, Y_train)
pred_svm = model_2.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6364583333333333
                  precision    recall  f1-score   support

        Analysis       0.41      0.42      0.42       137
      Conclusion       0.70      0.43      0.53        53
           Facts       0.71      0.88      0.79       483
         Invalid       0.55      0.42      0.48       165
           Issue       0.55      0.28      0.37        58
Rule/Law/Holding       0.57      0.27      0.36        64

        accuracy                           0.64       960
       macro avg       0.58      0.45      0.49       960
    weighted avg       0.62      0.64      0.61       960



In [174]:
x_un3 = tfidf_vect.transform(unlabel_3['Sentence'])
pred_unlabel_3 = model_2.predict_proba(x_un3)

pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_3:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9899271608727075, 0.996941140606315, 0.986809484173984, 0.9930678445003362, 0.9996558375526241, 0.9995396727808475, 0.9994222604403971, 0.9871660874383552, 0.9997945601840087, 0.9823207624245881, 0.9890051466771865, 0.9995801110861394, 0.9987716415324809, 0.9985064167781341, 0.987020342806131, 0.9986507652458673, 0.9994232992784667, 0.9886166405925358, 0.9989450741116748, 0.9984562535552618, 0.9899341608839298, 0.9892848573261409, 0.9984330622372584, 0.9920273289167408, 0.9910991967060971, 0.9816878005110007, 0.9969040799917905, 0.999039196151218, 0.9994917227115616, 0.9953225486912746, 0.9991829686021275, 0.9964362921744403, 0.9861130830547037, 0.9995681643975898, 0.9808876560829525, 0.9978212032319922, 0.9992013290388874, 0.9856679586881717, 0.9978677859256566, 0.9924257328404064]
[0, 2, 3, 4, 7, 8, 12, 14, 20, 22, 26, 27, 29, 30, 34, 35, 36, 40, 43, 46, 47, 51,

In [175]:
unlabel_3=unlabel_3.loc[[0, 2, 3, 4, 7, 8, 12, 14, 20, 22, 26, 27, 29, 30, 34, 35, 36, 40, 43, 46, 47, 51, 52, 53, 54, 56, 57, 63, 65, 67, 68, 70, 71, 72, 73, 75, 79, 81, 89, 96],:]
frame_3 = [train3_2,unlabel_3]
train3_3 = pd.concat(frame_3)

x_train_3 = tfidf_vect.transform(train3_3['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_3,train3_3['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC(multi_class='ovr',class_weight='balanced')
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_3=calibrator.fit(X_train, Y_train)
pred_svm = model_3.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6703755215577191
                  precision    recall  f1-score   support

        Analysis       0.49      0.37      0.42       103
      Conclusion       0.73      0.55      0.63        40
           Facts       0.75      0.88      0.81       375
         Invalid       0.51      0.52      0.52       108
           Issue       0.69      0.51      0.59        43
Rule/Law/Holding       0.46      0.26      0.33        50

        accuracy                           0.67       719
       macro avg       0.61      0.52      0.55       719
    weighted avg       0.65      0.67      0.65       719



In [176]:
x_un4 = tfidf_vect.transform(unlabel_4['Sentence'])
pred_unlabel_4 = model_3.predict_proba(x_un4)


pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_4:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[0, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 3, 2, 2, 3, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0.9933278515708008, 0.9885934325779983, 0.9900780646123866, 0.9982727236300323, 0.984939866016487, 0.9999338722825251, 0.9882821453478381, 0.9962265308875498, 0.9993792008475861, 0.9953793169703389, 0.9987843059782909, 0.9981691350913537, 0.9957202104452197, 0.9997456000803847, 0.9966354649235183, 0.9996547899684722, 0.9997267976624845, 0.999777789425247, 0.9985266837450577, 0.9968910716962864, 0.9961160407590917, 0.9998611982468466, 0.996939579193363, 0.9865811685844164, 0.9953575705527928, 0.9941178130690694, 0.9975356381213688, 0.9986361531567715, 0.9964696774016844, 0.9943068661295154, 0.994851011641429, 0.9865198974280731, 0.996069893281404, 0.9929382853911469, 0.9956195420862932, 0.9970682515302879, 0.9999617870460022, 0.9831407602101019, 0.9856106038504673, 0.994913788367826, 0.9804529779999647, 0.9857568290511572, 0.9985848315157997, 0.9984669

In [177]:
unlabel_4=unlabel_4.loc[[2, 4, 6, 7, 8, 10, 12, 15, 18, 21, 22, 27, 28, 29, 30, 31, 32, 34, 37, 38, 39, 40, 42, 46, 49, 50, 55, 56, 58, 60, 61, 66, 67, 69, 70, 71, 75, 80, 85, 87, 89, 90, 93, 97, 99 ],:]

In [178]:
frame_4 = [train3_3,unlabel_4]
train3_4 = pd.concat(frame_4)
len(train3_4)
x_train_4 = tfidf_vect.transform(train3_4['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_4,train3_4['Target'],test_size=0.27,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_4=calibrator.fit(X_train, Y_train)
pred_svm = model_4.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.6734486266531028
                  precision    recall  f1-score   support

        Analysis       0.45      0.36      0.40       138
      Conclusion       0.61      0.56      0.58        50
           Facts       0.78      0.87      0.82       537
         Invalid       0.49      0.51      0.50       144
           Issue       0.52      0.41      0.46        54
Rule/Law/Holding       0.57      0.35      0.43        60

        accuracy                           0.67       983
       macro avg       0.57      0.51      0.53       983
    weighted avg       0.66      0.67      0.66       983



In [179]:
x_un5 = tfidf_vect.transform(unlabel_5['Sentence'])

pred_unlabel_5 = model_4.predict_proba(x_un5)
pos=[]
large=[]
ind = []
i=0
for j in pred_unlabel_5:
  if max(j)> 0.98:
    ind.append(np.argmax(j))
    large.append(max(j))
    pos.append(i)
  i+=1


print(ind)
print(large)
print(pos)
print(len(ind))
print(len(large))
print(len(pos))

[2, 2, 5, 3, 2, 2, 2, 2, 2, 2, 1, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0]
[0.9991737892514412, 0.9973709842461448, 0.994593155027083, 0.9972488791233187, 0.9973019900817712, 0.9976972779822803, 0.9988458891295385, 0.9958130007512584, 0.9949975118428399, 0.9928393182459101, 0.9998807324561778, 0.9962344888653477, 0.9850644979627952, 0.9821382879837134, 0.9983011177033428, 0.9992479284961844, 0.9930855070866589, 0.9979757033002064, 0.9958562686115323, 0.997940807403725, 0.9898268722992031, 0.990428399062662, 0.9836514381921944, 0.9926954501153163, 0.9999001868657584, 0.9995301154991805, 0.9945876681663706, 0.9981901446568201, 0.9980811466091906, 0.9984333905556705, 0.9819069080088989, 0.9931506382281071, 0.9814635351810661, 0.9948229340237887]
[3, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 48, 50, 52, 53, 56, 60, 63, 65, 67, 75, 77, 78, 82, 86, 91, 93, 94, 95, 97, 100, 103, 110]
34
34
34


In [180]:
unlabel_5 = unlabel_5.loc[[3, 12, 18, 19, 22, 23, 26, 27, 28, 32, 36, 46, 48, 50, 52, 53, 56, 60, 63, 65, 67, 75, 77, 78, 82, 86, 91, 93, 94, 95, 97, 100, 103, 110],:]

In [181]:
frame_5 = [train3_4,unlabel_5]
train3_5 = pd.concat(frame_5)
len(train3_5)
x_train_5 = tfidf_vect.transform(train3_5['Sentence'])
X_train, x_val, Y_train, y_val = train_test_split(x_train_5,train3_5['Target'],test_size=0.2,random_state=42)
support = svm.LinearSVC()
clf = support.fit(X_train, Y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model_5=calibrator.fit(X_train, Y_train)
pred_svm = model_5.predict(x_val)
print('Accuracy %s' % accuracy_score(pred_svm,y_val))
print(classification_report(y_val,pred_svm))

Accuracy 0.672108843537415
                  precision    recall  f1-score   support

        Analysis       0.59      0.43      0.49       112
      Conclusion       0.66      0.50      0.57        38
           Facts       0.76      0.89      0.82       383
         Invalid       0.46      0.47      0.46       118
           Issue       0.50      0.35      0.41        34
Rule/Law/Holding       0.62      0.36      0.46        50

        accuracy                           0.67       735
       macro avg       0.60      0.50      0.54       735
    weighted avg       0.66      0.67      0.66       735



In [182]:
t_p = tfidf_vect.transform(test['Sentence'])
test_pred = model_5.predict(t_p)
print('Accuracy %s' % accuracy_score(test_pred,test['Target']))

Accuracy 0.7005758157389635
